In [ ]:
%pylab qt

In [ ]:
from PyQt4 import QtCore, QtGui
n_calc = [0]
def calculate():
    sin(rand(1000))
    n_calc[0]+=1
    timer.start()

timer = QtCore.QTimer()
timer.setInterval(0)
timer.setSingleShot(True)
timer.timeout.connect(calculate)

In [ ]:
from PyQt4 import QtCore, QtGui
from psutil import cpu_percent
from time import sleep
from timeit import default_timer
APP = QtGui.QApplication.instance()

close('all')

n_calc[0] = 0

from time import time
def sleep_loop(delay):
   loop = QtCore.QEventLoop()
   timer = QtCore.QTimer()
   timer.setInterval(delay*1000)
   timer.setSingleShot(True)
   timer.timeout.connect(loop.quit)
   timer.start()
   loop.exec() # la loop prend en charge elle-même l'évenement du timer qui va la faire mourir après delay.

def sleep_pe(delay):
   tic = default_timer()
   while(default_timer()<tic+delay):
       APP.processEvents()

def my_sleep(delay):
    tic = default_timer()
    if delay>1e-3:
        sleep_loop(delay - 1e-3)
    while(default_timer()<tic+delay):
        APP.processEvents()
        
import asyncio

async def sleep_coroutine(delay):
    await asyncio.sleep(delay)
    loop.stop()

loop = asyncio.get_event_loop()

def sleep_asyncio(delay):
    asyncio.ensure_future(sleep_coroutine(delay))
    loop.run_forever()
   
MAX_TIME = 0.005

for timer_on_off in (False, True):
    for index, (func, label) in enumerate([ (sleep_asyncio, "asyncio"),
                                            (sleep_loop, "qeventloop"),
                                            (sleep_pe, "processEvents"),
                                            (sleep, "time.sleep"),
                                            (my_sleep, "my_sleep"), 
                                           ]):
        if timer_on_off:
            timer.start()
        fig = figure()
        times_measured = []
        times_asked = []

        n_calc_before = n_calc[0]
        tic_for_func = default_timer()
        cpu_percent() # call starts the cpu_percent averaging
        for time_asked in linspace(0., MAX_TIME, 200):
            #if int(time_asked*10000)%10==0:
            #    print(time_asked)
            for i in range(10):
                tic = default_timer()
                func(time_asked)
                toc = default_timer()
                times_measured.append(toc - tic)
                times_asked.append(time_asked)
        cpu_percents = cpu_percent()
        toc_for_func = default_timer()
        calc_done = (n_calc[0] - n_calc_before)/(toc_for_func - tic_for_func)
        plot(times_asked, times_measured, '.')
        plot([0, MAX_TIME], [0, MAX_TIME])
        xlabel("time asked")
        ylabel("time measured")
        xlim(0, MAX_TIME)
        ylim(0, MAX_TIME)
        title(label + ' cpu_percent=' + str(cpu_percents) + ' calc/s=' + str(int(calc_done)))
        if timer_on_off:
            dirname = './images/with_timer/'
        else:
            dirname = './images/without_timer/'
        fig.savefig(dirname + label + '.png')
        timer.stop()
        close('all')
        for i in range(1000000):
            APP.processEvents()
timer.stop()

In [ ]:
%pylab qt
import quamash
from PyQt4 import QtCore, QtGui
import asyncio

from psutil import cpu_percent
from time import sleep
from timeit import default_timer
APP = QtGui.QApplication.instance()


n_calc = [0]
def calculate():
    sin(rand(1000))
    n_calc[0]+=1
    timer.start()

timer = QtCore.QTimer()
timer.setInterval(0)
timer.setSingleShot(True)
timer.timeout.connect(calculate)

MAX_TIME = 0.005
loop = quamash.QEventLoop()
asyncio.set_event_loop(loop) 

async def sleep_coroutine(delay):
    await asyncio.sleep(delay)
    #tic = default_timer()
    #if delay>0.001:
        #await asyncio.sleep(delay - 0.001)
    #while default_timer() < tic + delay:
    #    APP.processEvents()

timer.start()

label = "asyncio"
async def run():
    fig = figure()
    times_measured = []
    times_asked = []

    n_calc_before = n_calc[0]
    tic_for_func = default_timer()
    cpu_percent() # call starts the cpu_percent averaging
    for time_asked in linspace(0., MAX_TIME, 200):
        #if int(time_asked*10000)%10==0:
        #    print(time_asked)
        for i in range(10):
            tic = default_timer()
            await sleep_coroutine(time_asked)
            toc = default_timer()
            times_measured.append(toc - tic)
            times_asked.append(time_asked)
    cpu_percents = cpu_percent()
    toc_for_func = default_timer()
    calc_done = (n_calc[0] - n_calc_before)/(toc_for_func - tic_for_func)
    plot(times_asked, times_measured, '.')
    plot([0, MAX_TIME], [0, MAX_TIME])
    xlabel("time asked")
    ylabel("time measured")
    xlim(0, MAX_TIME)
    ylim(0, MAX_TIME)
    title(label + ' cpu_percent=' + str(cpu_percents) + ' calc/s=' + str(int(calc_done)))
    timer.stop()
    fig.savefig('asyncio_no_correction.png')

In [ ]:
asyncio.ensure_future(run())
#loop.run_forever() # this would be to launch the 

In [ ]:
%pylab qt
import asyncio
import scipy.fftpack
import quamash
from PyQt4 import QtCore, QtGui
import asyncio
loop = quamash.QEventLoop()
asyncio.set_event_loop(loop) 


class Scope(object):
    async def run_single(self, avg):
        y_avg = zeros(100)
        for i in range(avg):
            await asyncio.sleep(1)
            y_avg+=rand(100)
        return y_avg

class SpecAn(object):
    scope = Scope()
    
    async def run_single(self, avg):
        y = zeros(100, dtype=complex)
        for i in range(avg):
            trace = await self.scope.run_single(1)
            y+= scipy.fftpack.fft(trace)
        return y
            
sa = SpecAn()
scope = Scope()

In [ ]:
v = asyncio.ensure_future(sa.run_single(10))

In [ ]:
v.result()

In [ ]:
async def my_loop():
    for i in range(10):
        

In [ ]:
%pylab qt
import asyncio
import scipy.fftpack
import quamash
from PyQt4 import QtCore, QtGui
APP = QtGui.QApplication.instance()
import asyncio
from promise import Promise
loop = quamash.QEventLoop()
asyncio.set_event_loop(loop) 


class MyPromise(Promise):
    def get(self):
        while self.is_pending:
            APP.processEvents()
        return super(MyPromise, self).get()


class Scope(object):
    def __init__(self):
        self.timer = QtCore.QTimer()
        self.timer.setSingleShot(True)
        self.timer.setInterval(1000)
        self.timer.timeout.connect(self.check_for_curve)

    def run_single(self, avg):
        self.current_avg = 0
        self.avg = avg
        self.y_avg = zeros(100)
        self.p = MyPromise()
        self.timer.start()
        return self.p

    def check_for_curve(self):
        if self.current_avg<self.avg:
            self.y_avg += rand(100)
            self.current_avg += 1
            self.timer.start()
        else:
            self.p.fulfill(self.y_avg)


class SpecAn(object):
    scope = Scope()

    def __init__(self):
        timer = QtCore.QTimer()
        timer.setSingleShot(True)
        timer.setInterval(1000)

    def run_single(self, avg):
        self.avg = avg
        self.current_avg = 0
        self.y_avg = zeros(100, dtype=complex)
        p = self.scope.run_single(1)
        p.then(self.average_one_curve)
        self.p = MyPromise()
        return self.p

    def average_one_curve(self, trace):
        print('av')
        self.current_avg+=1
        self.y_avg+=scipy.fftpack.fft(trace)
        if self.current_avg>=self.avg:
            self.p.fulfill(self.y_avg)
        else:
            p = self.scope.run_single(1)
            p.then(self.average_one_curve)

sa = SpecAn()

In [ ]:
p = sa.run_single(3)

In [ ]:
p.is_fulfilled

In [ ]:
p.get()

In [ ]:
sa.avg

In [ ]:
s

In [23]:
%pylab qt
import asyncio
import scipy.fftpack
import quamash
from PyQt4 import QtCore, QtGui
import asyncio
loop = quamash.QEventLoop()
asyncio.set_event_loop(loop) 


class Scope(object):
    async def run_single(self, avg):
        y_avg = zeros(100)
        for i in range(avg):
            await asyncio.sleep(1)
            y_avg+=rand(100)
        return y_avg


class Na(object):
    async def run_single(self, avg):
        y_avg = zeros(100)
        for i in range(avg):
            await asyncio.sleep(1)
            y_avg+=rand(100)
        return y_avg

scope = Scope()
na = Na()

Populating the interactive namespace from numpy and matplotlib


In [24]:
async def my_coroutine(n):
    c1 = zeros(100)
    c2 = zeros(100)

    for i in range(n):
        print("launching f")
        f = asyncio.ensure_future(scope.run_single(1))
        print("launching g")
        g = asyncio.ensure_future(na.run_single(1))
        print("=======")
        c1+= await f
        c2+= await g
        print("f returned")
        print("g returned")

    return c1 + c2

In [25]:
p = asyncio.ensure_future(my_coroutine(3))

launching f
launching g
f returned
g returned
launching f
launching g
f returned
g returned
launching f
launching g
f returned
g returned


In [19]:
p.result()

array([ 2.06045693,  2.9179318 ,  2.99285774,  4.42323972,  2.75317522,
        3.62473716,  3.71125516,  1.735322  ,  2.64976214,  3.31580937,
        2.92686454,  2.28429018,  3.36139367,  2.32059774,  2.83418701,
        2.69074394,  2.19039055,  1.94149109,  3.05290117,  2.78291437,
        3.19430861,  3.2765257 ,  3.40944194,  3.02525901,  2.58175383,
        4.1429917 ,  2.8406369 ,  3.3005495 ,  2.76131675,  2.26251878,
        2.14933285,  3.54406952,  3.17758626,  2.87736436,  2.53237128,
        3.43577588,  3.67120035,  2.82549016,  3.12102662,  3.88246031,
        2.8460274 ,  2.04802908,  3.35135133,  2.97627906,  3.24942346,
        1.96564731,  3.85017412,  3.67758924,  2.86476129,  4.3136306 ,
        1.81448007,  2.63321507,  2.90885966,  1.71713038,  1.12230296,
        4.00504812,  2.81465738,  3.18881776,  2.93669662,  4.14013229,
        3.29903387,  2.48488255,  1.84620898,  3.09649301,  3.83207594,
        3.22461578,  3.17417224,  4.00152432,  2.75193527,  3.27

In [ ]:
p.result?

In [26]:
my_coroutine

<bound method Future.__iter__ of <Task finished coro=<my_coroutine() done, defined at <ipython-input-24-415a0bb4873c>:1> result=array([ 3.446...  3.25914873])>>

In [28]:
asyncio.ensure_future(my_coroutine(1))

<Task pending coro=<my_coroutine() running at <ipython-input-24-415a0bb4873c>:1>>

launching f
launching g
f returned
g returned


In [30]:
obj = my_coroutine(1)

In [32]:
obj.__await__

<method-wrapper '__await__' of coroutine object at 0x000001CA2B160830>

In [41]:
class MyAwaitable(object):
    def __await__(self):
        return ((x for x in range(5)))

In [44]:
m = MyAwaitable()

In [45]:
p = asyncio.ensure_future(m)

In [47]:
p.result()

RuntimeError: Task got bad yield: 0

In [52]:
m = MyFuture()

In [56]:
p = asyncio.ensure_future(m)

In [57]:
p.done()

False

In [58]:
m.set_result(5)

In [59]:
p

<MyFuture finished result=5>

In [60]:
p.result()

5

In [94]:
from asyncio.futures import Future

class MyFuture(Future):
    def __await__(self):
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(lambda : self.set_result(rand(100)))
        self.timer.setSingleShot(True)
        self.timer.setInterval(1000)
        self.timer.start()
        print('awaiting')
        
class AsyncScope(object):
    def run_single(self, avg):
        self.f = MyFuture()
        return self.f
    
a = AsyncScope()

In [83]:
f

<MyFuture pending>

In [75]:
f = a.run_single(3)

In [67]:
a.f.set_result(6)

In [68]:
f

<MyFuture finished result=6>

In [97]:
p = asyncio.ensure_future(a.run_single(3))

In [96]:
p.done()

False

In [100]:
p.__await__()

awaiting


In [101]:
p.result()

array([ 0.33594229,  0.59536305,  0.47270123,  0.73282475,  0.18595786,
        0.78269574,  0.65690543,  0.00856754,  0.3781265 ,  0.70170189,
        0.58660523,  0.26950735,  0.83684049,  0.09522667,  0.77477601,
        0.89018801,  0.53134898,  0.83955146,  0.13956472,  0.78778865,
        0.37838448,  0.94765879,  0.05130469,  0.85490265,  0.047397  ,
        0.81234497,  0.09754679,  0.20567203,  0.66152909,  0.81729331,
        0.24675232,  0.8831066 ,  0.80437425,  0.09911961,  0.28980736,
        0.6457419 ,  0.75490596,  0.68170566,  0.19117391,  0.07139178,
        0.54292759,  0.8990052 ,  0.92828782,  0.38504936,  0.53139293,
        0.30167843,  0.21626188,  0.95429376,  0.89264698,  0.2920714 ,
        0.91751696,  0.22926605,  0.29159616,  0.66521474,  0.3997597 ,
        0.59588074,  0.36224699,  0.00990077,  0.8124636 ,  0.73675774,
        0.47215568,  0.20821203,  0.35135282,  0.52779868,  0.54637853,
        0.80010143,  0.88699185,  0.47527497,  0.76666108,  0.74